In [1]:
from pathlib import Path
import requests

In [2]:
def download_raw_data(year: int, month:int) -> Path:
    """
    """
    URL = f"https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet"
    respose = requests.get(URL)

    if respose.status_code == 200:
        path = f'../data/raw/rides_{year}-{month:02d}.parquet'
        open(path, "wb").write(respose.content)
        return path
    else:
        raise Exception(f'{URL} is not available')

In [6]:
# download_raw_data(year=2022, month=1)

In [1]:
import pandas as pd

rides = pd.read_parquet("../data/raw/rides_2022-01.parquet")
rides.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [10]:
rides = rides[['tpep_pickup_datetime','PULocationID']]
rides.rename(columns={
    'tpep_pickup_datetime':'pickup_datetime',
    'PULocationID':'pickup_location_id'
}, inplace=True)

rides.head()

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68


In [12]:
rides['pickup_datetime'].describe()

count                          2463931
mean     2022-01-17 01:19:51.689726208
min                2008-12-31 22:23:09
25%                2022-01-09 15:37:41
50%                2022-01-17 12:11:45
75%         2022-01-24 13:49:37.500000
max                2022-05-18 20:41:57
Name: pickup_datetime, dtype: object

In [13]:
def validate_raw_data(
    rides: pd.DataFrame,
    year: int,
    month: int,
) -> pd.DataFrame:
    """
    Removes rows with pickup_datetimes outside their valid range
    """
    # keep only rides for this month
    this_month_start = f'{year}-{month:02d}-01'
    next_month_start = f'{year}-{month+1:02d}-01' if month < 12 else f'{year+1}-01-01'
    rides = rides[rides.pickup_datetime >= this_month_start]
    rides = rides[rides.pickup_datetime < next_month_start]
    
    return rides

In [14]:
validate_raw_data(rides, 2022, 1)

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68
...,...,...
2463926,2022-01-31 23:36:53,90
2463927,2022-01-31 23:44:22,107
2463928,2022-01-31 23:39:00,113
2463929,2022-01-31 23:36:42,148


In [17]:
rides = rides[rides.pickup_datetime >= '2022-01-01']
rides = rides[rides.pickup_datetime < '2022-02-01']
rides

,pickup_datetime,pickup_location_id
0,2022-01-01 00:35:40,142
1,2022-01-01 00:33:43,236
2,2022-01-01 00:53:21,166
3,2022-01-01 00:25:21,114
4,2022-01-01 00:36:48,68
...,...,...
2463926,2022-01-31 23:36:53,90
2463927,2022-01-31 23:44:22,107
2463928,2022-01-31 23:39:00,113
2463929,2022-01-31 23:36:42,148


In [16]:
rides['pickup_datetime'].describe()

count                          2463879
mean     2022-01-17 01:58:40.393673472
min                2022-01-01 00:00:08
25%                2022-01-09 15:37:56
50%                2022-01-17 12:11:54
75%                2022-01-24 13:49:37
max                2022-01-31 23:59:58
Name: pickup_datetime, dtype: object

In [12]:
rides.to_parquet('../data/transformed/validated_rides_2022_01.parquet')